In [ ]:
# default_exp client

# Client

> Contains the `DecadesClient` which is responsible for making calls to the [Spotify Web API](https://developer.spotify.com/documentation/web-api/) (through [spotipy](https://spotipy.readthedocs.io/en/2.16.1/#)) to control what music is playing on the radio.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastcore.all import *
from typing import *
import spotipy
from spotipy.oauth2 import SpotifyOAuth

## Authorization Code Flow

Make sure to set environment variables like so:
```
export SPOTIPY_CLIENT_ID='your-spotify-client-id'
export SPOTIPY_CLIENT_SECRET='your-spotify-client-secret'
export SPOTIPY_REDIRECT_URI='your-app-redirect-url'
```

In [ ]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

## Searching For Music

Search for music! You can change the `type` to search for playlists, tracks, etc. Read about the response [here](https://developer.spotify.com/documentation/web-api/reference/search/search/#writing-a-query---guidelines).

In [ ]:
res = sp.search("study jazz", limit=2, type="playlist")

In [ ]:
def get_first_playlist(query: str) -> str:
    search_results = sp.search(query, limit=1, type="playlist")
    first_item = search_results["playlists"]["items"][0]
    return {
        "name":        first_item["name"],
        "description": first_item["description"],
        "uri":         first_item["uri"]
    }

In [ ]:
get_first_playlist("1920s")

{'name': "1920s: The Roaring 20's🍾",
 'description': 'Welcome to the era of silent movies, prohibition, flappers, gangsters, naughty jazz and dirty blues! Photo: Clara Bow.',
 'uri': 'spotify:playlist:7olpTqGzGrYuI76ZBlCLgs'}

## Playback

**Note**: "spotify:(track):<code>" uri didn't work when playing songs on Spotify. It DID work when it was `track` or `playlist` though.

Spotify app must be opened somewhere for it to show up under devices. I am using [raspotify](https://dtcooper.github.io/raspotify/) to play Spotify on the Raspberry Pi. 

**To play music on Pi after initial turning on**:
   - Must first play music on pi through spotify connect

In [ ]:
devs = sp.devices(); devs

{'devices': [{'id': '55e742107c229068c72ff29dd37f3c6a6342411d',
   'is_active': True,
   'is_private_session': False,
   'is_restricted': False,
   'name': "Brandon's Raspberry Pi",
   'type': 'Speaker',
   'volume_percent': 38}]}

In [ ]:
#export
def get_device_id(devices: Dict, name: str):
    if len(devices["devices"]) == 0: raise ValueError("No active devices")
    for d in devices["devices"]:
        if d["name"] == name: 
            return d["id"]
    raise ValueError(f"No active device named {name}")

In [ ]:
get_device_id(sp.devices(), "Brandon's Raspberry Pi")

'55e742107c229068c72ff29dd37f3c6a6342411d'

Can pass [Spotify URI, URL, or ID](https://spotipy.readthedocs.io/en/2.16.1/?highlight=devices#ids-uris-and-urls) as `context_uri` to play songs/artists/etc. NOTE: **ID** actually didn't seem to work(?)

In [ ]:
sp.start_playback(device_id="55e742107c229068c72ff29dd37f3c6a6342411d",
                 context_uri="spotify:playlist:37i9dQZF1DX3SiCzCxMDOH")

## Creating MusicPlayer Class

Note, not sure if `device_id` changes over time. 

In [ ]:
{f"{str(year)}s": get_first_playlist(str(year))["uri"] for year in range(1920, 2020, 10)}

{'1920s': 'spotify:playlist:7olpTqGzGrYuI76ZBlCLgs',
 '1930s': 'spotify:playlist:6yY2tBY8976eFqsOIlBA0b',
 '1940s': 'spotify:playlist:245g9upOJ2BLTCcKeFyRDf',
 '1950s': 'spotify:playlist:5TLQjeDvm4igfsJgi6FaF7',
 '1960s': 'spotify:playlist:37i9dQZF1DXaKIA8E7WcJj',
 '1970s': 'spotify:playlist:37i9dQZF1DWTJ7xPn4vNaz',
 '1980s': 'spotify:playlist:37i9dQZF1DX4UtSsGT1Sbe',
 '1990s': 'spotify:playlist:37i9dQZF1DXbTxeAdrVG2l',
 '2000s': 'spotify:playlist:2f6tXtN0XesjONxicAzMIw',
 '2010s': 'spotify:playlist:3FeewjLi5LMzIpV4h35QEz'}

In [ ]:
#export
class DecadesClient:
    """Plays music on Raspberry Pi."""
    scope = 'user-read-private user-read-playback-state user-modify-playback-state'
    playlists = {
        '1920s': 'spotify:playlist:7olpTqGzGrYuI76ZBlCLgs',
        '1930s': 'spotify:playlist:6yY2tBY8976eFqsOIlBA0b',
        '1940s': 'spotify:playlist:245g9upOJ2BLTCcKeFyRDf',
        '1950s': 'spotify:playlist:5TLQjeDvm4igfsJgi6FaF7',
        '1960s': 'spotify:playlist:37i9dQZF1DXaKIA8E7WcJj',
        '1970s': 'spotify:playlist:37i9dQZF1DWTJ7xPn4vNaz',
        '1980s': 'spotify:playlist:37i9dQZF1DX4UtSsGT1Sbe',
        '1990s': 'spotify:playlist:37i9dQZF1DXbTxeAdrVG2l',
        '2000s': 'spotify:playlist:2f6tXtN0XesjONxicAzMIw',
        '2010s': 'spotify:playlist:3FeewjLi5LMzIpV4h35QEz'
    }

    def __init__(self, device_name: str):
        self.sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=DecadesClient.scope))
        self.device = get_device_id(self.sp.devices(), device_name)
    
    def play_music(self, decade: str):
        """Plays `decade` on device."""
        if decade not in self.playlists: raise KeyError(f"Invalid decade: {decade}")
        self.sp.start_playback(device_id=self.device, context_uri=DecadesClient.playlists[decade])
            
    def change_volume(self, volume: int): self.sp.volume(volume, device_id=self.device)

In [ ]:
player = DecadesClient("Brandon's Raspberry Pi")

In [ ]:
#export
@call_parse
def play(
    device_name: Param("Name of device", type=str)="Brandon's Raspberry Pi"
):
    """Creates a `DecadesPlayer` and forever loops controls :)."""
    player = DecadesClient(device_name)
    
    while True:
        action = input("What to change the (d)ecade or the (v)olume?")
        if action == "d": 
            decade = input("Which decade?")
            try: player.play_music(decade)
            except KeyError: print("Invalid decade")
        elif action == "v": 
            volume = int(input("What volume?"))
            player.change_volume(volume)
        else: print("Invalid choice")

In [ ]:
play("Brandon's Raspberry Pi")

What to change the (d)ecade or the (v)olume?d
Which decade?sdfajl
Invalid decade


KeyboardInterrupt: Interrupted by user

In [1]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_client.ipynb.
Converted 01_controls.ipynb.
Converted 02_player.ipynb.
Converted index.ipynb.
